Mistral with Gradio

In [ ]:
from huggingface_hub import login
login("Your_HUGGINGFACE_API_KEY")

In [ ]:
# Colab cell: install dependencies
!pip install --quiet sentence-transformers faiss-cpu transformers optimum

# Optional: peft for QAT and quantization
!pip install --quiet peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.8/425.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install faiss-cpu transformers sentence-transformers

In [ ]:
!pip install transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.4 MB/s eta 0:00:00


In [ ]:
import json
import torch
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

In [ ]:
# Colab cell: load context data
with open("/content/final.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

context_data = []
for section in raw_data.values():
    entries = section.get("entries", [])
    for entry in entries:
        text = entry.get("text")
        if text:
            context_data.append(text)

print(f"✅ Loaded {len(context_data)} context entries.")
print("Preview:", context_data[:2])


✅ Loaded 208 context entries.
Preview: ['Jawaharlal Nehru Government Engineering College, Sundernagar (JNGEC)', 'Sundernagar, Himachal Pradesh, India']


In [ ]:
embedding_model = SentenceTransformer("all-mpnet-base-v2")
context_embeddings = embedding_model.encode(context_data, convert_to_tensor=True, show_progress_bar=True)

dimension = context_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(context_embeddings.cpu().numpy())
print("✅ FAISS index built.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

✅ FAISS index built.


In [ ]:
# # Step 2: Build FAISS index
# dimension = context_embeddings.shape[1]
# index = faiss.IndexFlatL2(dimension)
# index.add(context_embeddings.cpu().numpy())

# Step 3: Load Mistral model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,), eps=1e-0

In [ ]:
def build_prompt(question, context):
    return f"""You are a helpful assistant for college website users. Use the context below to answer accurately and concisely. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Ask followup question for clarity of the question,Try to add emojies in answer to make the conversation interacive.

Context:
{context}

Question:
{question}
"""

def generate_answer(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            do_sample=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
def retrieve_context(query, k=2):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True).cpu().numpy()
    D, I = index.search(query_embedding.reshape(1, -1), k)
    retrieved_contexts = [context_data[i] for i in I[0]]
    return "\n---\n".join(retrieved_contexts)

In [ ]:
def chatbot_response(query):
    relevant_context = retrieve_context(query, k=3)
    prompt = build_prompt(query, relevant_context)
    answer = generate_answer(prompt)
    return answer

# Test with some example queries
test_questions = [
    "How many techers are ther in the college?"]

for q in test_questions:
    print(f"\nUser: {q}")
    print("Bot:", chatbot_response(q))


User: How many techers are ther in the college?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Bot: You are a helpful assistant for college website users. Use the context below to answer accurately and concisely. If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Ask followup question for clarity of the question,Try to add emojies in answer to make the conversation interacive.

Context:
There are over 1000 students enrolled.
---
The college has over 40 teachers.
---
The cources offered by the college are Mechanical, Electronics and Communication, Computer science, Textile and Civil Engineering.

Question:
How many techers are ther in the college?

Answer:
There are 40 teachers in the college. 👩‍🎓


In [ ]:
# Install Gradio (if not already)
!pip -q install gradio

import gradio as gr


In [ ]:

# Optional: clean wrapper to return only the model's final answer
def ask_bot(message, history):
    # message: latest user input
    # history: list of [user, bot] turns (you can use it if you want)
    # Use your existing pipeline
    answer = chatbot_response(message)
    # Enforce your instruction to thank the user in every answer if needed,
    # but your prompt already covers it. Here we just return the model output.
    return answer

# Build a ChatInterface
chat_ui = gr.ChatInterface(
    fn=ask_bot,
    title="College Chatbot",
    description="Ask questions about the college. Answers are generated from your RAG context and Mistral-7B.",
    examples=[
        "What is the name of the director of the college",
        "What are the admission requirements",
        "What courses are offered"
    ],
    theme="soft",
)

# Launch in Colab. share=True gives a public link in Colab.
chat_ui.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://313a5f8ae7de90da59.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
